In [1]:
import pandas as pd
from pathlib import Path

In [2]:
sample_candor_folder_path = '../data/candor/sample/0020a0c5-1658-4747-99c1-2839e736b481/'
# sample_transcribe_output_path = '../data/candor/sample/0020a0c5-1658-4747-99c1-2839e736b481/transcription/transcribe_output.json'

In [3]:
convo_path = Path(sample_candor_folder_path)
# convo_tokens = load_conversation_tokens(convo_path)

Backbiter transcript doesn't work as well for LM training purposes since turns are not split by full sentences, so sentences cannot be easily pieced back together.

Cliffhanger (below) is a better format since they are grouped by punctuation.

In [ ]:
# backbiter = pd.read_csv(convo_path / 'transcription/transcript_backbiter.csv')
# backbiter
# for id, row in backbiter.iterrows():
#     speaker = row['speaker'][-3:]
#     utterance = row['utterance']
#     print(f'{id=} {speaker=} {utterance=}')

In [11]:
cliffhanger = pd.read_csv(convo_path / 'transcription/transcript_cliffhanger.csv')
cliffhanger

,turn_id,speaker,start,stop,utterance,interval,delta,questions,end_question,overlap,n_words
0,0,5fa072f4f4aa580b63834357,4.34,190.04,"Mhm. Mhm. Just, mm. And Uh huh, mm. Mhm. Mhm. ...",NaN,185.70,2,False,False,38
1,1,5a73899f9cdd1800017786f0,198.34,201.76,Yeah hey I'm gone.,8.30,3.42,0,False,False,5
2,2,5fa072f4f4aa580b63834357,200.64,203.96,"Oh good, how are you?",-1.12,3.32,1,True,True,5
3,3,5a73899f9cdd1800017786f0,204.54,214.66,Yeah yeah I've done a few of these before they...,0.58,10.12,0,False,False,15
4,4,5fa072f4f4aa580b63834357,205.34,214.56,"Yeah. Yeah, so this will be an interesting stu...",-9.32,9.22,0,False,True,14
...,...,...,...,...,...,...,...,...,...,...,...
320,320,5fa072f4f4aa580b63834357,2727.21,2728.01,So thank you.,0.70,0.80,0,False,False,3
321,321,5a73899f9cdd1800017786f0,2728.29,2731.92,Yeah I get to enjoy your day evening.,0.28,3.63,0,False,False,8
322,322,5fa072f4f4aa580b63834357,2731.29,2733.00,"Yes, you too. Have a good night.",-0.63,1.71,0,False,True,7
323,323,5a73899f9cdd1800017786f0,2732.59,2734.09,Yeah. Yeah.,-0.41,1.50,0,False,True,2


In [13]:
for id, row in cliffhanger.iterrows():
    speaker = row['speaker'][-3:]
    utterance = row['utterance']
    print(f'{id=} {speaker=} {utterance=}')
    break

id=0 speaker='357' utterance='Mhm. Mhm. Just, mm. And Uh huh, mm. Mhm. Mhm. What? Mhm. Uh huh. Mhm. Yeah. Mhm. Mhm. Yeah. Oh. Oh yeah. Mhm. Yeah. Yeah. Great. Really? Yeah. Yeah. Yeah. Yeah. Yeah. Mhm. Yeah. Mhm. Yeah. Mhm. Yeah.'


## Load data from transcript_cliffhanger.csv with HuggingFace

In [15]:
from datasets import load_dataset, DatasetDict, load_from_disk

In [18]:
candor_sample_dir = '../data/candor/sample/0020a0c5-1658-4747-99c1-2839e736b481/'

In [19]:
candor_single_csv_data = load_dataset(
    "csv", 
    data_files=[candor_sample_dir + "transcription/transcript_cliffhanger.csv"]
)

Generating train split: 0 examples [00:00, ? examples/s]

In [20]:
candor_single_csv_data

DatasetDict({
    train: Dataset({
        features: ['turn_id', 'speaker', 'start', 'stop', 'utterance', 'interval', 'delta', 'questions', 'end_question', 'overlap', 'n_words'],
        num_rows: 325
    })
})

In [21]:
candor_single_csv_data['train']['utterance']

['Mhm. Mhm. Just, mm. And Uh huh, mm. Mhm. Mhm. What? Mhm. Uh huh. Mhm. Yeah. Mhm. Mhm. Yeah. Oh. Oh yeah. Mhm. Yeah. Yeah. Great. Really? Yeah. Yeah. Yeah. Yeah. Yeah. Mhm. Yeah. Mhm. Yeah. Mhm. Yeah.',
 "Yeah hey I'm gone.",
 'Oh good, how are you?',
 "Yeah yeah I've done a few of these before they're not too bad.",
 "Yeah. Yeah, so this will be an interesting study. Mhm. I don't know.",
 'Is this your first time doing?',
 "Okay, this is actually my first one so uh yeah it's my first time doing yeah chad study so I guess I don't even know when there are, so it's kind of cool. There we go.",
 'Uh huh yep not too bad.',
 "Oh excellent, so hard that we've been going, I don't know.",
 "Uh Feel a little crappy today but Uh huh it's that time of year I suppose so yeah the kid's been a little sick yeah the kid hasn't been feeling good and I think she shared it with me so but yeah.",
 "Oh no, are you getting sick? Oh thanks, Tiny firm sponge. I swear that's that's what kids do, they're just 

In [23]:
def put_data_into_splits(data, train=0.8, val=0.1, test=0.1):
    # (.train_test_split shuffles by default)
    train_valtest = data['train'].train_test_split(test_size = 1 - train)
    test_valid = train_valtest['test'].train_test_split(test_size = test / (val + test))
    out = DatasetDict({
            'train': train_valtest['train'],
            'val': test_valid['train'],
            'test': test_valid['test']
        })
    return out

In [24]:
example_split_test = put_data_into_splits(candor_single_csv_data)
example_split_test

DatasetDict({
    train: Dataset({
        features: ['turn_id', 'speaker', 'start', 'stop', 'utterance', 'interval', 'delta', 'questions', 'end_question', 'overlap', 'n_words'],
        num_rows: 260
    })
    val: Dataset({
        features: ['turn_id', 'speaker', 'start', 'stop', 'utterance', 'interval', 'delta', 'questions', 'end_question', 'overlap', 'n_words'],
        num_rows: 32
    })
    test: Dataset({
        features: ['turn_id', 'speaker', 'start', 'stop', 'utterance', 'interval', 'delta', 'questions', 'end_question', 'overlap', 'n_words'],
        num_rows: 33
    })
})

## Tokenize CANDOR

In [28]:
from transformers import GPT2TokenizerFast, GPT2Config, GPT2LMHeadModel
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

In [29]:
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
context_length = 1024

In [30]:
print("Vocabulary size:", tokenizer.vocab_size)
print("Max Model Input Sizes:", tokenizer.model_max_length)
print("Padding token:", tokenizer.pad_token)
print("Special tokens:", tokenizer.all_special_tokens)

Vocabulary size: 50257
Max Model Input Sizes: 1024
Padding token: None
Special tokens: ['<|endoftext|>']


In [31]:
tokenized_data_path = '../data/candor/tokens_sentence/sample/'

In [32]:
# Tokenize:
encoded_datasets = example_split_test.map(
    lambda sentence: tokenizer(
        sentence['utterance'],
        truncation=True,
        max_length=context_length,
        return_overflowing_tokens=True,
        return_length=True,
        ), 
    batched=True)
encoded_datasets.save_to_disk(tokenized_data_path)

Map:   0%|          | 0/260 [00:00<?, ? examples/s]

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

Map:   0%|          | 0/33 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/260 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/32 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/33 [00:00<?, ? examples/s]

In [33]:
# Load pretokenized data:
encoded_datasets = load_from_disk(tokenized_data_path)

In [35]:
print(encoded_datasets)
print(encoded_datasets['train'][0])
# input_ids is key column

DatasetDict({
    train: Dataset({
        features: ['turn_id', 'speaker', 'start', 'stop', 'utterance', 'interval', 'delta', 'questions', 'end_question', 'overlap', 'n_words', 'input_ids', 'attention_mask', 'length', 'overflow_to_sample_mapping'],
        num_rows: 260
    })
    val: Dataset({
        features: ['turn_id', 'speaker', 'start', 'stop', 'utterance', 'interval', 'delta', 'questions', 'end_question', 'overlap', 'n_words', 'input_ids', 'attention_mask', 'length', 'overflow_to_sample_mapping'],
        num_rows: 32
    })
    test: Dataset({
        features: ['turn_id', 'speaker', 'start', 'stop', 'utterance', 'interval', 'delta', 'questions', 'end_question', 'overlap', 'n_words', 'input_ids', 'attention_mask', 'length', 'overflow_to_sample_mapping'],
        num_rows: 33
    })
})
{'turn_id': 213, 'speaker': '5a73899f9cdd1800017786f0', 'start': 2113.43, 'stop': 2128.75, 'utterance': "And I was like well we don't know what our test results are, I don't want to go hand out

## Finetune pretrained model on CANDOR

In [ ]:
trained_model = GPT2LMHeadModel.from_pretrained("gpt2")

In [ ]:
import accelerate
import transformers

transformers.__version__, accelerate.__version__

In [ ]:
import torch
torch.cuda.is_available()
torch.cuda.get_device_name(0)

In [ ]:
training_output_dir = '../models/candor_finetuned/test1'
args = TrainingArguments(
    training_output_dir,
    per_device_train_batch_size=32, # change to fit GPU specs
    per_device_eval_batch_size=32,
    evaluation_strategy='epoch',
    eval_steps=0.25,
    logging_steps=0.25,
    save_strategy='epoch',
    save_steps=0.25,
    group_by_length=True, # bucketing
    load_best_model_at_end=True,
    metric_for_best_model='loss',
    greater_is_better=False,
    save_total_limit=3,
    num_train_epochs=1,
)

In [ ]:
args.device

In [ ]:
trainer = Trainer(
    model=trained_model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['val'],
)

In [ ]:
trainer.train(
    # resume_from_checkpoint=True
)